In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import normalize

warnings.filterwarnings('ignore')

from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Flatten
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau


from sklearn.model_selection import train_test_split

from sklearn import metrics

import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) Client VM (build 25.201-b09, mixed mode, sharing)
  Starting server from C:\Users\anton\AppData\Roaming\Python\Python36\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\anton\AppData\Local\Temp\tmpwrizhnut
  JVM stdout: C:\Users\anton\AppData\Local\Temp\tmpwrizhnut\h2o_anton_started_from_python.out
  JVM stderr: C:\Users\anton\AppData\Local\Temp\tmpwrizhnut\h2o_anton_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,03 secs
H2O cluster timezone:,Europe/Athens
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.11
H2O cluster version age:,3 months and 11 days !!!
H2O cluster name:,H2O_from_python_anton_12i9nr
H2O cluster total nodes:,1
H2O cluster free memory:,247.5 Mb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


## First we deal with the dataset "last year"

In [3]:
df=pd.read_csv("last year.csv")

In [4]:
hf=h2o.H2OFrame(df)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
hf

SIZE-3year,GR-PROFIT-MARGIN-3year,CAPITAL-EMPLOYED-TURNOVER-3year,STOCKHOLDERS-EQUITY-TURNOVER-3year,CAPITAL-EMPLOYED/NET-FIXED-ASSETS-3year,DEBT/EQUITY-3year,EQUITY/CAPITAL-EMPLOYED-3year,WORKING-CAPITAL/TOTAL-ASSETS-3year,AV-COLLECTION-PERIOD-FOR-RECIEVABLES-3year,AV-PAYMENT-PERIOD-3year,AV-TURNOVER-PERIOD-FOR-INVENTORIES-3year,SCALED-CHANGE-ASSETS-2year,SIZE-2year,CH.NET-INCOME-2year,GR-PROFIT-MARGIN-2year,CAPITAL-EMPLOYED-TURNOVER-2year,STOCKHOLDERS-EQUITY-TURNOVER-2year,CAPITAL-EMPLOYED/NET-FIXED-ASSETS-2year,DEBT/EQUITY-2year,EQUITY/CAPITAL-EMPLOYED-2year,WORKING-CAPITAL/TOTAL-ASSETS-2year,AV-COLLECTION-PERIOD-FOR-RECIEVABLES-2year,AV-PAYMENT-PERIOD-2year,AV-TURNOVER-PERIOD-FOR-INVENTORIES-2year,SCALED-CHANGE-ASSETS,SIZE,CH.NET-INCOME,GR-PROFIT-MARGIN,CAPITAL-EMPLOYED-TURNOVER,STOCKHOLDERS-EQUITY-TURNOVER,CAPITAL-EMPLOYED/NET-FIXED-ASSETS,DEBT/EQUITY,EQUITY/CAPITAL-EMPLOYED,WORKING-CAPITAL/TOTAL-ASSETS,AV-COLLECTION-PERIOD-FOR-RECIEVABLES,AV-PAYMENT-PERIOD,AV-TURNOVER-PERIOD-FOR-INVENTORIES,final
17.0484,11.394,8.55332,9.78249,0.545139,22.9557,0.87435,-0.338667,431.196,307.434,59.9585,-0.163349,16.7187,-0.332261,0.007019,4.43754,5.07321,1.76599,16.5084,0.874702,0.0109,698.487,326.597,0,0.044032,16.8068,-1,11.8664,2.79988,3.51854,1.09679,35.1517,0.795749,-0.024809,2233.51,939.705,0,Bankrupted
17.7774,5.02206,0.405121,0.405121,1.33546,0.676128,1,0.073301,61.8285,134.43,365.416,0.028336,17.8341,-0.472987,20.1219,0.455641,0.500191,1.12506,0.804943,0.910933,0.008382,50.3083,276.609,366.879,-0.097722,17.638,-1,-48.8533,1.8397,2.40263,0.459472,3.11552,0.765702,-0.494313,33.6375,129.11,45.7495,Bankrupted
11.8552,6.37271,2.93941,2.93941,50.0545,2.08991,1,0.192623,188.645,29.3107,109.392,0.319564,12.5176,1,17.3665,1.58542,1.58543,378.147,1.05019,0.999993,0.427538,283.19,195.588,89.9187,0.329882,13.2029,-1,-0.0022,3.9575,3.9575,41.1074,3.72599,1,0.171188,359.853,317.256,17.1971,Bankrupted
13.9112,16.4816,5.66139,5.66136,13.3043,6.36057,1.00001,0.101952,357.85,239.631,0,0.036113,13.9834,0.298251,20.037,5.80123,5.80123,12.6862,6.6823,1,0.0906,271.615,288.644,0,0.061063,14.1057,0.040065,18.0594,7.0608,7.0608,20.4195,7.69199,1,0.082127,310.991,204.567,0,Bankrupted
14.934,24.8771,6.29598,6.29598,0.855904,5.91638,1,-0.079627,232.387,173.792,58.0486,0.054719,15.0435,-1,20.1757,7.0686,7.0686,0.731521,7.84956,1,-0.118476,241.826,183.836,57.2869,-0.276377,14.476,-0.639322,18.532,7.52303,7.523,0.54607,8.23642,1,-0.277293,70.5558,167.958,16.6845,Bankrupted
13.7423,38.3074,6.12232,6.12232,0.526064,5.63726,1,-0.283839,11.8471,245.601,0,0.298616,14.3583,-0.351794,38.8719,5.69165,5.69161,0.448397,10.4069,1.00001,-0.218278,240.976,418.328,0,0.218348,14.8021,0.666632,26.8184,4.98607,4.98607,1.10773,7.3506,1,-0.083698,154.362,133.217,0.132322,Bankrupted
15.12,19.4367,6.70298,6.70298,63.7685,5.35213,1,0.14074,311.934,29.9276,32.6707,-0.083622,14.9524,0.492204,15.3583,5.24937,5.24937,79.2801,4.25741,1,0.173942,325.461,42.9279,30.3994,-0.147615,14.655,-0.830522,16.8676,3.56988,3.56988,78.6983,3.2293,1,0.217872,340.371,100.051,62.5733,Bankrupted
15.4624,41.9848,2.92924,3.44413,8.89938,5.07154,0.850502,0.14212,395.91,202.878,176.986,0.15242,15.7696,-0.092149,40.9129,1.52127,2.59767,3.2981,4.51277,0.585629,0.193179,457.893,349.874,209.19,0.10167,15.9737,-0.74118,40.4265,1.57713,2.73252,5.36249,5.78108,0.57717,0.194968,481.964,410.512,345.177,Bankrupted
14.943,33.9506,3.96655,4.10605,0.779867,2.93403,0.966026,-0.133341,165.189,26.0325,63.5665,0.12061,15.1854,0.071923,33.9974,1.15739,4.54815,2.46172,3.3325,0.254475,0.458061,151.134,27.9052,70.4715,0.05871,15.303,0.072768,45,1.36344,4.46509,1.46244,3.3467,0.305355,0.110541,104.575,122.932,91.1567,Bankrupted
18.0841,-71.7875,0.021042,0.038628,2.81772,1.14337,0.544736,0.521513,2089.15,1686.27,771.431,-0.007257,18.0695,0.941226,38.7468,0.031585,0.058735,2.87307,1.18658,0.537763,0.520988,1920.42,1359.94,122.14,-0.00695,18.0556,-0.742006,59.5347,0.034124,0.064885,2.81814,1.25897,0.525912,0.510865,1717.9,1794.89,0,

## Train test split

In [6]:
np.random.seed(333)

train,test,valid = hf.split_frame(ratios=[.70,.15])


## H2O auto ml model

In [7]:
model = H2OAutoML(max_models=15,seed=1)

model.train(x=train.columns[:-1],y=train.columns[-1],training_frame=train,validation_frame=valid)

AutoML progress: |████████████████████████████████████████████████████████| 100%


## Performance

In [12]:
lb=model.leaderboard
print(lb.head())

y_pred=model.leader.predict(test)

model_id,auc,logloss,mean_per_class_error,rmse,mse
DRF_1_AutoML_20200317_103945,0.825137,0.503227,0.246812,0.408698,0.167034
GBM_2_AutoML_20200317_103945,0.802368,0.526341,0.226548,0.419324,0.175832
GBM_1_AutoML_20200317_103945,0.800091,0.561703,0.224044,0.423142,0.179049
GBM_3_AutoML_20200317_103945,0.79326,0.525713,0.377505,0.422492,0.178499
GBM_grid_1_AutoML_20200317_103945_model_2,0.792805,0.60816,0.270719,0.458001,0.209765
GBM_4_AutoML_20200317_103945,0.759563,0.563391,0.254326,0.434804,0.189054
GBM_grid_1_AutoML_20200317_103945_model_1,0.754098,0.653705,0.278916,0.480275,0.230664
XRT_1_AutoML_20200317_103945,0.746812,0.56956,0.292805,0.44095,0.194437
GBM_grid_1_AutoML_20200317_103945_model_4,0.739071,1.05583,0.438752,0.499567,0.249567
GBM_grid_1_AutoML_20200317_103945_model_3,0.677596,0.637906,0.486111,0.472804,0.223544



drf prediction progress: |████████████████████████████████████████████████| 100%


In [28]:
perf = model.leader.model_performance(test)
auc_last_year=perf.auc()
print(auc_last_year)

0.6755952380952381


## Dataset  "1 years before"

In [31]:
df2=pd.read_csv("1 years before.csv")

In [32]:
hf=h2o.H2OFrame(df2)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [33]:
hf

SIZE-3year,GR-PROFIT-MARGIN-3year,CAPITAL-EMPLOYED-TURNOVER-3year,STOCKHOLDERS-EQUITY-TURNOVER-3year,CAPITAL-EMPLOYED/NET-FIXED-ASSETS-3year,DEBT/EQUITY-3year,EQUITY/CAPITAL-EMPLOYED-3year,WORKING-CAPITAL/TOTAL-ASSETS-3year,AV-COLLECTION-PERIOD-FOR-RECIEVABLES-3year,AV-PAYMENT-PERIOD-3year,AV-TURNOVER-PERIOD-FOR-INVENTORIES-3year,SCALED-CHANGE-ASSETS-2year,SIZE-2year,CH.NET-INCOME-2year,GR-PROFIT-MARGIN-2year,CAPITAL-EMPLOYED-TURNOVER-2year,STOCKHOLDERS-EQUITY-TURNOVER-2year,CAPITAL-EMPLOYED/NET-FIXED-ASSETS-2year,DEBT/EQUITY-2year,EQUITY/CAPITAL-EMPLOYED-2year,WORKING-CAPITAL/TOTAL-ASSETS-2year,AV-COLLECTION-PERIOD-FOR-RECIEVABLES-2year,AV-PAYMENT-PERIOD-2year,AV-TURNOVER-PERIOD-FOR-INVENTORIES-2year,final
17.0484,11.394,8.55332,9.78249,0.545139,22.9557,0.87435,-0.338667,431.196,307.434,59.9585,-0.163349,16.7187,-0.332261,0.007019,4.43754,5.07321,1.76599,16.5084,0.874702,0.0109,698.487,326.597,0,Bankrupted
17.7774,5.02206,0.405121,0.405121,1.33546,0.676128,1,0.073301,61.8285,134.43,365.416,0.028336,17.8341,-0.472987,20.1219,0.455641,0.500191,1.12506,0.804943,0.910933,0.008382,50.3083,276.609,366.879,Bankrupted
11.8552,6.37271,2.93941,2.93941,50.0545,2.08991,1,0.192623,188.645,29.3107,109.392,0.319564,12.5176,1,17.3665,1.58542,1.58543,378.147,1.05019,0.999993,0.427538,283.19,195.588,89.9187,Bankrupted
13.9112,16.4816,5.66139,5.66136,13.3043,6.36057,1.00001,0.101952,357.85,239.631,0,0.036113,13.9834,0.298251,20.037,5.80123,5.80123,12.6862,6.6823,1,0.0906,271.615,288.644,0,Bankrupted
14.934,24.8771,6.29598,6.29598,0.855904,5.91638,1,-0.079627,232.387,173.792,58.0486,0.054719,15.0435,-1,20.1757,7.0686,7.0686,0.731521,7.84956,1,-0.118476,241.826,183.836,57.2869,Bankrupted
13.7423,38.3074,6.12232,6.12232,0.526064,5.63726,1,-0.283839,11.8471,245.601,0,0.298616,14.3583,-0.351794,38.8719,5.69165,5.69161,0.448397,10.4069,1.00001,-0.218278,240.976,418.328,0,Bankrupted
15.12,19.4367,6.70298,6.70298,63.7685,5.35213,1,0.14074,311.934,29.9276,32.6707,-0.083622,14.9524,0.492204,15.3583,5.24937,5.24937,79.2801,4.25741,1,0.173942,325.461,42.9279,30.3994,Bankrupted
15.4624,41.9848,2.92924,3.44413,8.89938,5.07154,0.850502,0.14212,395.91,202.878,176.986,0.15242,15.7696,-0.092149,40.9129,1.52127,2.59767,3.2981,4.51277,0.585629,0.193179,457.893,349.874,209.19,Bankrupted
14.943,33.9506,3.96655,4.10605,0.779867,2.93403,0.966026,-0.133341,165.189,26.0325,63.5665,0.12061,15.1854,0.071923,33.9974,1.15739,4.54815,2.46172,3.3325,0.254475,0.458061,151.134,27.9052,70.4715,Bankrupted
18.0841,-71.7875,0.021042,0.038628,2.81772,1.14337,0.544736,0.521513,2089.15,1686.27,771.431,-0.007257,18.0695,0.941226,38.7468,0.031585,0.058735,2.87307,1.18658,0.537763,0.520988,1920.42,1359.94,122.14,Bankrupted


## Train test split

In [35]:
np.random.seed(333)

train,test,valid = hf.split_frame(ratios=[.70,.15])

## H2O auto ml model

In [36]:
model2 = H2OAutoML(max_models=15,seed=1)

model2.train(x=train.columns[:-1],y=train.columns[-1],training_frame=train,validation_frame=valid)

AutoML progress: |████████████████████████████████████████████████████████| 100%


## Performance

In [37]:
lb=model2.leaderboard
print(lb.head())

y_pred=model2.leader.predict(test)

model_id,auc,logloss,mean_per_class_error,rmse,mse
DeepLearning_grid_1_AutoML_20200317_105401_model_1,0.651046,2.07644,0.445946,0.551928,0.304624
GBM_grid_1_AutoML_20200317_105401_model_4,0.621622,1.30542,0.472973,0.557721,0.311053
XRT_1_AutoML_20200317_105401,0.619878,0.675172,0.472973,0.485943,0.23614
GBM_4_AutoML_20200317_105401,0.605057,0.705389,0.5,0.500587,0.250588
DeepLearning_grid_1_AutoML_20200317_105401_model_2,0.601351,1.79717,0.459459,0.594975,0.353995
GBM_1_AutoML_20200317_105401,0.588056,0.850803,0.5,0.528686,0.279509
DRF_1_AutoML_20200317_105401,0.576286,0.71435,0.5,0.502556,0.252562
GBM_3_AutoML_20200317_105401,0.57367,0.712831,0.472973,0.504408,0.254428
GBM_grid_1_AutoML_20200317_105401_model_2,0.572799,0.656852,0.5,0.482005,0.232329
GBM_2_AutoML_20200317_105401,0.560157,0.744803,0.5,0.518037,0.268362



deeplearning prediction progress: |███████████████████████████████████████| 100%


In [39]:
perf = model2.leader.model_performance(test)
auc_1year=perf.auc()
print(auc_1year)

0.6587301587301587


## Dataset "2 years before"

In [40]:
df3=pd.read_csv("2 years before.csv")

In [41]:
hf=h2o.H2OFrame(df3)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [44]:
hf

SIZE-3year,GR-PROFIT-MARGIN-3year,CAPITAL-EMPLOYED-TURNOVER-3year,STOCKHOLDERS-EQUITY-TURNOVER-3year,CAPITAL-EMPLOYED/NET-FIXED-ASSETS-3year,DEBT/EQUITY-3year,EQUITY/CAPITAL-EMPLOYED-3year,WORKING-CAPITAL/TOTAL-ASSETS-3year,AV-COLLECTION-PERIOD-FOR-RECIEVABLES-3year,AV-PAYMENT-PERIOD-3year,AV-TURNOVER-PERIOD-FOR-INVENTORIES-3year,final
17.0484,11.394,8.55332,9.78249,0.545139,22.9557,0.87435,-0.338667,431.196,307.434,59.9585,Bankrupted
17.7774,5.02206,0.405121,0.405121,1.33546,0.676128,1,0.073301,61.8285,134.43,365.416,Bankrupted
11.8552,6.37271,2.93941,2.93941,50.0545,2.08991,1,0.192623,188.645,29.3107,109.392,Bankrupted
13.9112,16.4816,5.66139,5.66136,13.3043,6.36057,1.00001,0.101952,357.85,239.631,0,Bankrupted
14.934,24.8771,6.29598,6.29598,0.855904,5.91638,1,-0.079627,232.387,173.792,58.0486,Bankrupted
13.7423,38.3074,6.12232,6.12232,0.526064,5.63726,1,-0.283839,11.8471,245.601,0,Bankrupted
15.12,19.4367,6.70298,6.70298,63.7685,5.35213,1,0.14074,311.934,29.9276,32.6707,Bankrupted
15.4624,41.9848,2.92924,3.44413,8.89938,5.07154,0.850502,0.14212,395.91,202.878,176.986,Bankrupted
14.943,33.9506,3.96655,4.10605,0.779867,2.93403,0.966026,-0.133341,165.189,26.0325,63.5665,Bankrupted
18.0841,-71.7875,0.021042,0.038628,2.81772,1.14337,0.544736,0.521513,2089.15,1686.27,771.431,Bankrupted


## Train test split

In [45]:
np.random.seed(333)

train,test,valid = hf.split_frame(ratios=[.70,.15])

## H2O auto ml model

In [46]:
model3 = H2OAutoML(max_models=15,seed=1)

model3.train(x=train.columns[:-1],y=train.columns[-1],training_frame=train,validation_frame=valid)

AutoML progress: |████████████████████████████████████████████████████████| 100%


## Performance

In [47]:
lb=model3.leaderboard
print(lb.head())

y_pred=model3.leader.predict(test)

model_id,auc,logloss,mean_per_class_error,rmse,mse
DeepLearning_grid_1_AutoML_20200317_105913_model_1,0.701102,1.60451,0.386364,0.544937,0.296957
DeepLearning_1_AutoML_20200317_105913,0.689164,0.63588,0.469697,0.467939,0.218967
DRF_1_AutoML_20200317_105913,0.671947,0.602255,0.469697,0.458545,0.210264
GBM_grid_1_AutoML_20200317_105913_model_2,0.668733,0.608788,0.424242,0.45836,0.210094
DeepLearning_grid_1_AutoML_20200317_105913_model_2,0.65955,1.23429,0.363636,0.525186,0.27582
GBM_3_AutoML_20200317_105913,0.651286,0.633496,0.484848,0.472524,0.223279
DeepLearning_grid_1_AutoML_20200317_105913_model_3,0.641873,0.874137,0.401515,0.519969,0.270368
GBM_1_AutoML_20200317_105913,0.635216,0.772219,0.469697,0.514568,0.26478
XRT_1_AutoML_20200317_105913,0.631772,0.629383,0.462121,0.473114,0.223836
GBM_2_AutoML_20200317_105913,0.625115,0.639168,0.5,0.475494,0.226095



deeplearning prediction progress: |███████████████████████████████████████| 100%


In [48]:
perf = model3.leader.model_performance(test)
auc_2year=perf.auc()
print(auc_2year)

0.76875


## Results

In [49]:
data={'H2O AUTOML':[auc_1year,auc_2year,auc_last_year]}
    

results=pd.DataFrame(data,index=['1 years before','2 years before','last year'])

results

,H2O AUTOML
1 years before,0.658730
2 years before,0.768750
last year,0.675595
